In [1]:
# python manage.py shell_plus --notebook

import os
import django

from random import randint

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

import pandas as pd
from django.core.files.uploadedfile import UploadedFile
from wfrp.models import (Campaign, Item, Talent, Spell, AdvancedSkill,
                         PlayableCharacter, Armour, Weapon, Creature, CreatureTrait)
from django.conf import settings
from users.models import User
from wfrp.scripts.default_content import (addDefaultCharacter,
                                          addDefaultCreatures,
                                          addDefaultCampaign)


In [2]:
EXAMPLE_MEDIA_ROOT = '../example_media/'

### Purge current instances

In [3]:
Campaign.objects.all().delete()
PlayableCharacter.objects.all().delete()
Item.objects.all().delete()
Armour.objects.all().delete()
Talent.objects.all().delete()
Spell.objects.all().delete()
AdvancedSkill.objects.all().delete()
Weapon.objects.all().delete()
# User.objects.all().delete()
User.objects.all().exclude(username='admin').delete()
Creature.objects.all().delete()
CreatureTrait.objects.all().delete()


(1, {'wfrp.CreatureTrait': 1})

### Create test users

In [4]:
users = [
         {'name': 'gunnar'   , 'character':'GUNNAR'},
         {'name': 'else'     , 'character':'ELSE'},
         {'name': 'amris'    , 'character':'AMRIS'},
         {'name': 'ferdinand', 'character':'FERDINAND'},
         {'name': 'molrella' , 'character':'MOLRELLA'},
         {'name': 'master'   , 'character':''}
        ]

for user in users:
    user_instance = User.objects.create_user(username=user['name'],
                                             password=f'{user["name"]}123')
    user_instance.save()

admin = User.objects.filter(username='admin')[0]
# admin = User.objects.create_user('admin', password='123')
# admin.is_superuser = True
# admin.is_staff = True
# admin.save()

### Create default campaign

In [5]:
campaign_master = User.objects.get(username='master')
campaign_name = 'Doing the rounds'
campaign_description = '''
The Characters, strolling through the Ubersreik marketplace, are caught up in a riot and later charged with instigating it. Mysteriously, a local lawyer intervenes and convinces the judge in charge of the case to allow the Characters to pay their 'debt' by working as members of the Guard.

Soon, the group finds themselves patrolling the streets of Ubersreik with Rudi Klumpenklug, a thoroughly corrupt sergeant of the Guard. The temporary guards are exposed to various crimes, none of which Klumpenklug shows any interest in pursuing, leaving the Characters to resolve, ignore or even exploit each situation as they prefer.

Finally, Ilse Fassenwütend contacts the group. Fassenwütend is a road guard, who claims to be able to have the Characters' sentences commuted if they help her escort a criminal to the place where he is to be executed.

However, it will not be an easy task. The group will have a horrific night ahead of them, leading a terrified man through strangely altered streets while Morrslieb, the Chaos moon, is in the crescent (and therefore shining in the sky) and mutant cultists attack them from all sides. Will the Characters manage to protect the man, who claims to be innocent, and then take him to the scaffold? Or will they choose a different path?
'''

campaign = addDefaultCampaign(campaign_master,
                              campaign_name,
                              campaign_description,
                              EXAMPLE_MEDIA_ROOT)


### Add default characters

In [6]:
for user_entry in users:
    if len(user_entry['character']) > 0:
        user = User.objects.filter(username=user_entry['name'])[0]
        campaign = Campaign.objects.all()[0]
        addDefaultCharacter(user=user, 
                            campaign=campaign,
                            character_name=user_entry['character'], 
                            EXAMPLE_MEDIA_ROOT=EXAMPLE_MEDIA_ROOT)

### Add default creatures

In [7]:
addDefaultCreatures(EXAMPLE_MEDIA_ROOT)